## Web Scraping and Writing to Google Sheets
    1. Get the data and parse with requests, lxml and Beatiful Soup 
    2. Data wrange into dictionaries 
    3. Create Pandas DF and clean the data
    4. Write to Google Sheets 

In [1]:
from requests import get 
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup 

In [2]:
#imputing URL and creating header creds for parsing 
url = 'http://pfref.com/tiny/J65i5'

my_header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}


In [3]:
#grabbing the HTML and getting text 
fantasy_page = get(url, headers = my_header)

doc = lh.fromstring(fantasy_page.content)

In [4]:
print(fantasy_page)

<Response [200]>


Response == 200 means the page scraped successfully 

In [5]:
#parsing table elements in the HTML inside the pattern "//tr" --> this is a table element 

table_elements = doc.xpath('//tr')

In [15]:
[len(T) for T in table_elements[:-1]]

len(table_elements[1])


20

In [7]:
#creating an empty array to insert the table elements 
cols = []
i = 0 #setting the increment 

for t in table_elements[1]:
    i+1
    name = t.text_content() #getting the column name from the HTML table 
    #print('%d:"%s"'% (i, name))
    cols.append((name, [])); 

In [12]:
table_elements[1].text_content()

'\n         Rk\n         Player\n         Pos\n         Age\n         Date\n         Lg\n         Tm\n         \n         Opp\n         Result\n         G#\n         Week\n         Day\n         Tgt\n         Rec\n         Yds\n         Y/R\n         TD\n         Ctch%\n         Y/Tgt\n      '

In [16]:
#Since out first row is the header, data is stored on the second row onwards

for j in range(1,len(table_elements)):
    #T is our j'th row
    T=table_elements[j]
    
    #If row is not of size 24, the //tr data is not from our table 
    if len(T)!=len(table_elements[1]):
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 

        #Append the data to the empty list of the i'th column
        cols[i][1].append(data)
        #Increment i for the next column
        i+=1

In [11]:
#[len(C) for (title,C) in cols]

In [17]:
#creating a dictionary for the columns in the parsed table 
Dict={title:column for (title,column) in cols}

df=pd.DataFrame(Dict)

In [18]:
df

,Rk,Player,Pos,Age,Date,Lg,Tm,,Opp,Result,G#,Week,Day,Tgt,Rec,Yds,Y/R,TD,Ctch%,Y/Tgt
0,Rk,Player,Pos,Age,Date,Lg,Tm,,Opp,Result,G#,Week,Day,Tgt,Rec,Yds,Y/R,TD,Ctch%,Y/Tgt
1,1,Michael Gallup,WR,23.227,2019-11-17,NFL,DAL,@,DET,W 35-27,10,11,Sun,13,9,148,16.44,0,69.2%,11.38
2,2,Calvin Ridley,WR,24.332,2019-11-17,NFL,ATL,@,CAR,W 29-3,10,11,Sun,8,8,143,17.88,1,100.0%,17.88
3,3,John Brown,WR,29.228,2019-11-17,NFL,BUF,@,MIA,W 37-20,10,11,Sun,14,9,137,15.22,2,64.3%,9.79
4,4,DeVante Parker,WR,26.301,2019-11-17,NFL,MIA,,BUF,L 20-37,10,11,Sun,10,7,135,19.29,0,70.0%,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,96,LeSean McCoy,RB,31.129,2019-11-18,NFL,KAN,@,LAC,W 24-17,11,11,Mon,6,4,28,7.00,0,66.7%,4.67
101,97,Eric Ebron,TE,26.221,2019-11-17,NFL,IND,,JAX,W 33-13,10,11,Sun,4,4,27,6.75,0,100.0%,6.75
102,98,Trey Edmunds,TE,24.319,2019-11-14,NFL,PIT,@,CLE,L 7-21,10,11,Thu,3,3,27,9.00,0,100.0%,9.00
103,99,Patrick Ricard,FB/DL,25.174,2019-11-17,NFL,BAL,,HOU,W 41-7,10,11,Sun,3,3,27,9.00,0,100.0%,9.00


### Light data cleaning 
    1. Dropping the "Rk" row because it is empty 
    2. Dropping a blank column used for formatting in the HTML 

In [19]:
df = df.drop(df.loc[df["Rk"] == 'Rk'].index)
df = df.drop('', 1)

In [20]:
#Grapping Parameters for looping 
n_rows = df.shape[0]
n_cols = df.shape[1]

In [21]:
#import library
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('quickstart/g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)

In [22]:
# creating a new sheet to drop the data 
#new_sheet = client.create('target_sheet')

#new_sheet.share('players@fantasy-data-build.iam.gserviceaccount.com', perm_type='user', role='writer')

In [23]:
#sheet = client.open('target_sheet')

#ws = sheet.get_worksheet(0)

In [27]:
#Now will can access our google sheets we call client.open on StartupName
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/11D57-HeQYujW7vZtN0MXsGpyNgWj45Q87BPiAD9CNCk/edit#gid=0') #2019-q4_fantasy-web-scraping/passing

ws = sheet.get_worksheet(0)

#Access all of the record inside that
#result = sheet.get_all_record()

shaped_data = df.transpose()

ws.insert_row(df.columns.tolist(), 1)

for i in range(1, n_rows): 
    row = df.iloc[i-1].tolist()
    index = i+1
    if index == 50:
            time.sleep(60)
    ws.insert_row(row, index)
    print(i)


APIError: {
  "error": {
    "code": 429,
    "message": "Quota exceeded for quota group 'WriteGroup' and limit 'Write requests per user per 100 seconds' of service 'sheets.googleapis.com' for consumer 'project_number:686366856477'.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Google developer console API key",
            "url": "https://console.developers.google.com/project/686366856477/apiui/credential"
          }
        ]
      }
    ]
  }
}
